In [1]:
%pip install pandas
%pip install requests
%pip install osm2geojson
%pip install geopandas
%pip install pyproj








Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [41]:
import argparse
import json
from pprint import pprint
from typing import Dict

import requests

NOMINATIM_API_URL = "https://nominatim.openstreetmap.org"
NOMINATIM_DETAILS_ENDPOINT = f"{NOMINATIM_API_URL}/details"
NOMINATIM_SEARCH_ENDPOINT = f"{NOMINATIM_API_URL}/search"
NOMINATIM_REVERSE_ENDPOINT = f"{NOMINATIM_API_URL}/reverse"

# query_params = {
#     "namedetails": 1,
#     "polygon_geojson": 1,
#     "hierarchy": 1,
# }

query_params = {
    "namedetails": 1,
    "hierarchy": 1,
}


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("command", type=str, choices=["details", "reverse", "search"])
    return parser.parse_args()


def fetch_osm_details(osm_id: str, osm_type: str, params: Dict[str, int]) -> dict:
    params_query = "&".join(f"{param_name}={param_value}" for param_name, param_value in params.items())
    request_url = f"{NOMINATIM_DETAILS_ENDPOINT}?osmtype={osm_type}&osmid={osm_id}&{params_query}&format=json"
    print(request_url)
    response = requests.get(request_url)
    response.raise_for_status()
    return response.json()

def fetch_osm_search(query: str, params: Dict[str, int]) -> dict:
    params_query = "&".join(f"{param_name}={param_value}" for param_name, param_value in params.items())
    request_url = f"{NOMINATIM_SEARCH_ENDPOINT}?q={query}&{params_query}&format=json"
    print(request_url)
    response = requests.get(request_url)
    response.raise_for_status()
    return response.json()

def search_for_city_by_zip(zip_code: str) -> dict:
    request_url = f"{NOMINATIM_SEARCH_ENDPOINT}?country=usa&postalcode={zip_code}&format=json"
    print(request_url)
    response = requests.get(request_url)
    response.raise_for_status()
    return response.json()


def get_population_by_city_state_country(city: str, state: str, country: str) -> dict:
    request_url = f"{NOMINATIM_SEARCH_ENDPOINT}?city={city}&state={state}&country={country}&format=json"
    print(request_url)
    response = requests.get(request_url)
    response.raise_for_status()
    result = response.json()
    result = result[0]
    return response.json()
    



# find the city name given that zip code
# find the population of that city
# determine if the population is greater than gaza's population 
# if not, store the population and find the bordering cities
# find the population of the bordering cities
# determine which combination of cities and their percentages would equal that of gazas population


In [46]:
def geocode_city_state_country(city: str, state: str, country: str) -> dict:
    request_url = f"{NOMINATIM_SEARCH_ENDPOINT}?city={city}&state={state}&country={country}&format=json"
    print(request_url)
    response = requests.get(request_url)
    response.raise_for_status()
    result = response.json()
    result = result[0]
    # address = result['address']
    return result

def geocode_address(street: str, city: str, state: str) -> dict:
    request_url = f"{NOMINATIM_SEARCH_ENDPOINT}?street={street}&city={city}&state={state}&format=json"
    request_url = request_url.replace(" ", "%20") # escape url
    print(request_url)
    response = requests.get(request_url)
    response.raise_for_status()
    result = response.json()
    result = result[0]
    # address = result['address']
    return result

# result = geocode_address(street="30 Maujer st", city="Brooklyn", state="NY")
# result = geocode_address(street="1228 fairmont st", city="washington", state="dc")
geocode_result = geocode_address(street="19 Richard ln", city="bloomfield", state="ct")
latlon = (geocode_result['lat'], geocode_result['lon'])
pprint(latlon)


https://nominatim.openstreetmap.org/search?street=19%20Richard%20ln&city=bloomfield&state=ct&format=json
('41.8496006', '-72.7370846')


In [47]:
# This function is used to find the place_id of a geocoded location
def fetch_osm_reverse(lat: float, lon: float, params: Dict[str, int], zoom: int = 10) -> dict:
    params_query = "&".join(f"{param_name}={param_value}" for param_name, param_value in params.items())
    # request_url = f"{NOMINATIM_REVERSE_ENDPOINT}?lat={lat}&lon={lon}&zoom={zoom}&{params_query}&format=geocodejson"
    request_url = f"{NOMINATIM_REVERSE_ENDPOINT}?lat={lat}&lon={lon}&zoom={zoom}&{params_query}&format=json"
    print(request_url)
    response = requests.get(request_url)
    response.raise_for_status()
    result = response.json()
    # result = result[0]
    return result

reverse = fetch_osm_reverse(lat=latlon[0], lon=latlon[1], params=query_params)
print(reverse)

# This function is used to find the place_id of a geocoded location
def get_type_id_city_from_reverse_data(reverse):
    try:
        if reverse and reverse.get('osm_type') in ['relation', 'way']:
                        entity_type_char = reverse['osm_type'][0].upper()
                        city = reverse['address'].get('city', reverse['address'].get('town'))
                        if city:
                            return (entity_type_char, reverse['osm_id'], city)
    except:
        print('no city found')
    return None

result = get_type_id_city_from_reverse_data(reverse)
print(result)


https://nominatim.openstreetmap.org/reverse?lat=41.8496006&lon=-72.7370846&zoom=10&namedetails=1&hierarchy=1&format=json
{'place_id': 11364911, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 5970033, 'lat': '41.826488', 'lon': '-72.7300945', 'class': 'boundary', 'type': 'administrative', 'place_rank': 16, 'importance': 0.4147752048189652, 'addresstype': 'town', 'name': 'Bloomfield', 'display_name': 'Bloomfield, Capitol Planning Region, Connecticut, 06002, United States', 'address': {'town': 'Bloomfield', 'county': 'Capitol Planning Region', 'state': 'Connecticut', 'ISO3166-2-lvl4': 'US-CT', 'postcode': '06002', 'country': 'United States', 'country_code': 'us'}, 'namedetails': {'name': 'Bloomfield'}, 'boundingbox': ['41.8004219', '41.9069105', '-72.7987639', '-72.6859091']}
('R', 5970033, 'Bloomfield')


In [50]:

def create_overpass_query(osm_id: str, osm_type: str):
    if osm_type not in ['R', 'W']:
        raise ValueError("Invalid osm_type. Must be 'R' or 'W'.")
    # if osm_type is R set string to relation else set string to way
    osm_type = "relation" if osm_type == 'R' else "way"
    overpass_query = f"""
    [out:json][timeout:25];
    {osm_type}({osm_id});
    (._;>;);
    out body;
    """
    print(overpass_query)
    return overpass_query

query_str = create_overpass_query(osm_id=result[1], osm_type=result[0])
overpass_url = "http://overpass-api.de/api/interpreter"
response = requests.get(overpass_url, params={'data': query_str})
overpass_data = response.json()
print(overpass_data)


    [out:json][timeout:25];
    relation(5970033);
    (._;>;);
    out body;
    
{'version': 0.6, 'generator': 'Overpass API 0.7.61.5 4133829e', 'osm3s': {'timestamp_osm_base': '2023-12-28T16:51:13Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'node', 'id': 158863474, 'lat': 41.826488, 'lon': -72.7300945, 'tags': {'census:population': '20611;2006', 'ele': '40', 'gnis:feature_id': '205590', 'name': 'Bloomfield', 'place': 'town', 'population': '20611', 'wikidata': 'Q542675'}}, {'type': 'node', 'id': 378322291, 'lat': 41.9069105, 'lon': -72.7589894}, {'type': 'node', 'id': 378322361, 'lat': 41.9065067, 'lon': -72.7593101}, {'type': 'node', 'id': 378322396, 'lat': 41.9058736, 'lon': -72.7599675}, {'type': 'node', 'id': 378322411, 'lat': 41.9053856, 'lon': -72.7605415}, {'type': 'node', 'id': 378322514, 'lat': 41.9051096, 'lon': -72.7612838}, {'type': 'node', 'id': 378322596, 'lat': 41.90

In [52]:
import osm2geojson
import geopandas as gpd
from pyproj import Proj, transform, CRS


def get_utm_epsg_code(given_lat, given_lon):
    """
    Returns the EPSG code of the UTM zone for a given latitude and longitude using pyproj.
    """
    # Create a geographic coordinate system based on the WGS84 datum (EPSG:4326)
    geodetic = CRS('EPSG:4326')
    
    # Projected coordinate system: UTM
    utm = CRS(geodetic.to_epsg(lat=given_lat, lon=given_lon))
    return utm.to_string()

# Function to extract polygon coordinates from Overpass data
def extract_polygon_coordinates(overpass_data):
    nodes = {element['id']: (element['lon'], element['lat'])
             for element in overpass_data['elements']
             if element['type'] == 'node'}
    relation = next(el for el in overpass_data['elements'] if el['type'] == 'relation')
    coordinates = [nodes[member['ref']] for member in relation['members'] if member['type'] == 'node']
    return coordinates


polygon_coords = extract_polygon_coordinates(overpass_data)
print(overpass_data)

# epsg_code = get_utm_epsg_code(given_lat=latlon[0], given_lon=latlon[1])
# print(epsg_code)
# geojson_data = osm2geojson.json2geojson(overpass_data)
# # print(geojson_data)

# gdf = gpd.GeoDataFrame.from_features(geojson_data['features'], crs='EPSG:4326')
# df_proj = gdf.to_crs('EPSG:32610')

# print(gdf.area)
# gdf.plot()

# area = gdf.area[0]
# print(area)

{'version': 0.6, 'generator': 'Overpass API 0.7.61.5 4133829e', 'osm3s': {'timestamp_osm_base': '2023-12-28T16:51:13Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'node', 'id': 158863474, 'lat': 41.826488, 'lon': -72.7300945, 'tags': {'census:population': '20611;2006', 'ele': '40', 'gnis:feature_id': '205590', 'name': 'Bloomfield', 'place': 'town', 'population': '20611', 'wikidata': 'Q542675'}}, {'type': 'node', 'id': 378322291, 'lat': 41.9069105, 'lon': -72.7589894}, {'type': 'node', 'id': 378322361, 'lat': 41.9065067, 'lon': -72.7593101}, {'type': 'node', 'id': 378322396, 'lat': 41.9058736, 'lon': -72.7599675}, {'type': 'node', 'id': 378322411, 'lat': 41.9053856, 'lon': -72.7605415}, {'type': 'node', 'id': 378322514, 'lat': 41.9051096, 'lon': -72.7612838}, {'type': 'node', 'id': 378322596, 'lat': 41.9039934, 'lon': -72.762686}, {'type': 'node', 'id': 378322628, 'lat': 41.9034498, 'lon